In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
# 텐서플로우에 기본 내장된 mnist 모듈을 이용하여 데이터를 로드합니다.
# 지정한 폴더에 MNIST 데이터가 없는 경우 자동으로 데이터를 다운로드합니다.
# one_hot 옵션은 레이블을 동물 분류 예제에서 보았던 one_hot 방식의 데이터로 만들어줍니다.
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')
# 입력 값의 차원은 [배치크기, 특성값] 으로 되어 있습니다.
# 손글씨 이미지는 28x28 픽셀로 이루어져 있고, 이를 784개의 특성값으로 정합니다. develop
X = tf.placeholder(tf.float32, [None, 784])
# 결과는 0~9 의 10 가지 분류를 가집니다.
Y = tf.placeholder(tf.float32, [None, 10])

# 신경망의 레이어는 다음처럼 구성합니다.
# 784(입력 특성값)
#   -> 256 (히든레이어 뉴런 갯수) -> 256 (히든레이어 뉴런 갯수)
#   -> 10 (결과값 0~9 분류)
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01), name='W1')
    # 입력값에 가중치를 곱하고 ReLU 함수를 이용하여 레이어를 만듭니다.
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01), name='W2')
    # L1 레이어의 출력값에 가중치를 곱하고 ReLU 함수를 이용하여 레이어를 만듭니다.
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01), name='W3')
    # 최종 모델의 출력값은 W3 변수를 곱해 10개의 분류를 가지게 됩니다.
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)
    
with tf.name_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    


In [4]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        # 텐서플로우의 mnist 모델의 next_batch 함수를 이용해
        # 지정한 크기만큼 학습할 데이터를 가져옵니다.
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost = total_cost+cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
    summary, acc = sess.run([merged, accuracy], feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    writer.add_summary(summary, global_step = sess.run(global_step))
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
# for epoch in range(15):
#     total_cost = 0;
    
#     for i in range(total_batch):
#         # 텐서플로우의 mnist 모델의 next_batch 함수를 이용해
#         # 지정한 크기만큼 학습할 데이터를 가져옵니다
#         batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
#         _, cost_val = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys})
#         total_cost += cost_val
        
# #     print('Epoch:', '%04d' % (epoch+1),
# #          'Avg. cost =', '%04f'% (total_cost / total_batch))
    
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.413
Epoch: 0002 Avg. cost = 0.153
Epoch: 0003 Avg. cost = 0.099
Epoch: 0004 Avg. cost = 0.072
Epoch: 0005 Avg. cost = 0.052
Epoch: 0006 Avg. cost = 0.041
Epoch: 0007 Avg. cost = 0.032
Epoch: 0008 Avg. cost = 0.024
Epoch: 0009 Avg. cost = 0.021
Epoch: 0010 Avg. cost = 0.016
Epoch: 0011 Avg. cost = 0.015
Epoch: 0012 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.015
Epoch: 0014 Avg. cost = 0.014
Epoch: 0015 Avg. cost = 0.012
최적화 완료!


In [5]:
#########
# 결과 확인
######
# model 로 예측한 값과 실제 레이블인 Y의 값을 비교합니다.
# tf.argmax 함수를 이용해 예측한 값에서 가장 큰 값을 예측한 레이블이라고 평가합니다.
# 예) [0.1 0 0 0.7 0 0.2 0 0 0 0] -> 3
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images,
                                   Y: mnist.test.labels}))


정확도: 0.9779
